In [1]:
import pandas as pd
import os
import io
import numpy as np
import numpy as np
import re
import time
import pandas as pd
import string
from collections import Counter, defaultdict
from sklearn import metrics
import xml.etree.ElementTree as ET
from sklearn.svm import LinearSVC
import nltk
from nltk.stem.snowball import FrenchStemmer 
from nltk.corpus import stopwords
from sklearn.calibration import CalibratedClassifierCV
from unicodedata import normalize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

def read_w2v(language='english'):
    if(language == 'french'):
        path = 'wiki.multi.fr.vec'
    elif(language == 'english'):
        path = 'wiki.multi.en.vec'
    elif(language == 'german'):
        path = 'wiki.multi.de.vec'
    t0 = time.time()
    w2v = {}
    count = 0
    
    with open("/Users/vjstark/Downloads/ADM stuff/"+ path, "r", encoding="utf8") as lines:
        for line in lines:
            lineArr = line.split()
            if(count!=0):
                x = []
                for value in lineArr[len(lineArr)-300:]:
                        x.append(float(value))
                w2v[' '.join(lineArr[0:len(lineArr)-300])]=  np.array(x)
            count+=1
    print(count)
    print(time.time()-t0)
    return w2v

def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

negation = {'arent','isnt','wasnt','werent','cant','couldnt','mustnt','shouldnt','wont','wouldnt','didnt','doesnt','dont','hasnt','havent','hadnt'}

def tokenize(doc):    
    doc = doc.lower()
    token_list = doc.split()
    tokenized_list=[]
    for token in token_list:
        new_token=''
        for i in range(0,len(token)):
            if(token[i] not in string.punctuation):
                new_token+=token[i]
        tokenized_list.append(new_token)
    return tokenized_list

def cleaningTextTokenizing(line, lang = 'english'):
    if (lang == 'english'):
        re_print = re.compile('[^%s]' % re.escape(string.printable))
        line = normalize('NFD', line).encode('ascii', 'ignore')
        line = line.decode('UTF-8')
        line = tokenize(line)
        line = [re_print.sub('', w) for w in line]
        line = [word for word in line if word.isalpha()]
    else:
        line = tokenize(line)
        line = [word for word in line if word.isalpha()]
    return line

# Stop word removal of tokenized input data
def get_lemmatized_text(tokenized_review):
    lemmatizer = WordNetLemmatizer()
    lemmatizedStr = []
    for word in tokenized_review:
        lemmatizedStr.append(lemmatizer.lemmatize(word))
    return lemmatizedStr

def data_parse(parsed_xml_data, language):
    X_data = []
    y_data = []
#     X_refer = []
    for node in parsed_xml_data.getroot():
        
        summary = node.find('summary')
        rating = node.find('rating')
        text = node.find('text')
        tokens_summary = getvalueofnode(summary)
        tokens_text = getvalueofnode(text)
        if(tokens_summary == None and tokens_text == None ):
            tokens = []
        elif(tokens_text == None):
            tokens = cleaningTextTokenizing(tokens_summary,language)
        elif(tokens_summary == None):
            tokens = cleaningTextTokenizing(tokens_text,language)
        else:
            tokens = [*cleaningTextTokenizing(tokens_summary,language), *cleaningTextTokenizing(tokens_text,language)]
        
        lemmaStr = get_lemmatized_text(tokens)
        X_data.append(lemmaStr)
#     X_refer.append(tokens_summary + " " + tokens_text)
        if(float(getvalueofnode(rating))>3):
            y_data.append('positive')
        else:
            y_data.append('negative')
        
    return X_data, y_data

def words_not_w2vec(vocab, w2v):
    new_words = []
    for word,i in list(vocab):
        if(word not in w2v):
            new_words.append(word)
    return new_words

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x, min_df = 0.001)
        tfidf.fit(X)
        print ('Vocab Size' , len(tfidf.vocabulary_))
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self,tfidf.vocabulary_.items()

    def transform(self, X):
        np_ar = np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])
        return np_ar

In [3]:
#x_path = 'C:/Users/sunny/Desktop/adm project/amazon dataset/cls-acl10-unprocessed/'
x_path = '/Users/vjstark/Downloads/ADM stuff/cls-acl10-unprocessed/'
#ENGLISH
parsed_xml = ET.parse(x_path+'en/books/train.review')
parsed_xml_test = ET.parse(x_path+'en/books/test.review')
parsed_xml_unlabeled = ET.parse(x_path+'en/books/unlabeled.review')
#FRENCH
parsed_xml_train_fr = ET.parse(x_path+'fr/books/train.review')
parsed_xml_test_fr = ET.parse(x_path+'fr/books/test.review')
parsed_xml_unlabeled_fr = ET.parse(x_path+'fr/books/unlabeled.review')
#GERMAN
parsed_xml_train_de = ET.parse(x_path+'de/books/train.review')
parsed_xml_test_de = ET.parse(x_path+'de/books/test.review')
parsed_xml_unlabeled_de = ET.parse(x_path+'de/books/unlabeled.review')

In [4]:
w2v = read_w2v('english')

200001
18.81849503517151


In [5]:
w2v_fr = read_w2v('french')

200001
19.066630125045776


In [6]:
w2v_de = read_w2v('german')

200001
19.474637031555176


In [7]:
#English
X, y = data_parse(parsed_xml,'english')
X_test, y_test = data_parse(parsed_xml_test,'english')
X_unlabeled, y_unlabeled = data_parse(parsed_xml_unlabeled,'english')

In [8]:
#French
X_train_fr, y_train_fr = data_parse(parsed_xml_train_fr,'french')
X_test_fr, y_test_fr = data_parse(parsed_xml_test_fr,'french')
X_unlabeled_fr, y_unlabeled_fr = data_parse(parsed_xml_unlabeled_fr, 'french')

In [9]:
#German
X_train_de, y_train_de = data_parse(parsed_xml_train_de,'german')
X_test_de, y_test_de = data_parse(parsed_xml_test_de,'german')
X_unlabeled_de, y_unlabeled_de = data_parse(parsed_xml_unlabeled_de, 'german')

In [28]:
print(f'EN-ul: {len(X_unlabeled)}, FR-ul: {len(X_unlabeled_fr)}, DE-ul: {len(X_unlabeled_de)}')

EN-ul: 50000, FR-ul: 32870, DE-ul: 165470


In [50]:
#len(X_unlabeled)
len(y_unlabeled)

50000

In [61]:
cpos = 0
cneg = 0
for i in y_unlabeled:
    if i == 'positive':
        cpos += 1
    else:
        cneg += 1
        
print(cpos, cneg)

25000 25000


In [10]:
def vectorize_predict(w2v_lang, X_data, y_data,clf):
    t0 = time.time()
    vectorizer_data = TfidfEmbeddingVectorizer(w2v_lang)
    X ,vocab = vectorizer_data.fit(X_data)
    X_vect_data = vectorizer_data.transform(X_data)
    result = clf.predict(X_vect_data)
    print ('Accuracy Score ', metrics.accuracy_score(y_data, result))
    print ('F1 Score ',metrics.f1_score(y_data, result,average="binary", pos_label="negative"))
    print ('Precision Score ',metrics.precision_score(y_data, result,average="binary", pos_label="negative"))
    print ('Recall Score ',metrics.recall_score(y_data, result,average="binary", pos_label="negative"))
    print(time.time()-t0)
    return words_not_w2vec(vocab,w2v_lang)

### En-En, En-Fr, En-De

In [11]:
from sklearn.pipeline import Pipeline

vectorizer = TfidfEmbeddingVectorizer(w2v)
vect,vocab = vectorizer.fit(X_unlabeled)
X_t = vectorizer.transform(X_unlabeled)
clfLSVC = LinearSVC()
LSVC = CalibratedClassifierCV(clfLSVC)
unknown_words_list_en = words_not_w2vec(vocab,w2v)
print(len(unknown_words_list_en))

Vocab Size 6996
60


In [12]:
LSVC.fit(X_t, y_unlabeled)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid')

In [13]:
unknown_words_test_en = vectorize_predict(w2v, X_test,y_test, LSVC)
len(unknown_words_test_en)

# Vocab Size 6996
# Accuracy Score  0.81882
# F1 Score  0.8223063494242953
# Precision Score  0.8067818790654709
# Recall Score  0.83844
# 31.686428785324097
#60

Vocab Size 9113
Accuracy Score  0.8405
F1 Score  0.842779694430754
Precision Score  0.8309037900874635
Recall Score  0.855
3.30194091796875


247

In [63]:
unknown_words_test_fr = vectorize_predict(w2v_fr, X_test_fr,y_test_fr, LSVC)
len(unknown_words_test_fr)
# Vocab Size 6261
# Accuracy Score  0.7598722239123821
# F1 Score  0.7549594858899132
# Precision Score  0.7707276876267748
# Recall Score  0.7398235473075753
# 21.79134178161621
# 534

Vocab Size 8933
Accuracy Score  0.5715
F1 Score  0.2668947818648418
Precision Score  0.9230769230769231
Recall Score  0.156
2.461233377456665


1057

In [64]:
unknown_words_test_de = vectorize_predict(w2v_de, X_test_de,y_test_de, LSVC)
len(unknown_words_test_de)

# Vocab Size 7575
# Accuracy Score  0.7439596301444371
# F1 Score  0.7789966771516356
# Precision Score  0.6852286909918508
# Recall Score  0.9024959207107028
# 93.78028178215027
# 100

Vocab Size 11213
Accuracy Score  0.7565
F1 Score  0.7821029082774049
Precision Score  0.7076923076923077
Recall Score  0.874
4.295766115188599


634

### Fr-Fr, Fr-En, Fr-De

In [17]:
vectorizer = TfidfEmbeddingVectorizer(w2v_fr)
vect,vocab = vectorizer.fit(X_unlabeled_fr)
X_t = vectorizer.transform(X_unlabeled_fr)
clfLSVC = LinearSVC()
LSVC = CalibratedClassifierCV(clfLSVC)
unknown_words_list_fr = words_not_w2vec(vocab,w2v_fr)
print(len(unknown_words_list_fr))

Vocab Size 6261
534


In [18]:
LSVC.fit(X_t, y_unlabeled_fr)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid')

In [19]:
unknown_words_test_en = vectorize_predict(w2v, X_test,y_test, LSVC)
len(unknown_words_test_en)

# Vocab Size 6996
# Accuracy Score  0.51658
# F1 Score  0.5918922113225387
# Precision Score  0.5121103222602039
# Recall Score  0.70112
# 37.17140984535217
# 60

Vocab Size 9113
Accuracy Score  0.7365
F1 Score  0.7786644267114657
Precision Score  0.6712527154236061
Recall Score  0.927
2.850640296936035


247

In [20]:
unknown_words_test_fr = vectorize_predict(w2v_fr, X_test_fr,y_test_fr, LSVC)
len(unknown_words_test_fr)

# Vocab Size 6261
# Accuracy Score  0.52299969577122
# F1 Score  0.6233635206226429
# Precision Score  0.5150035722791141
# Recall Score  0.7894736842105263
# 14.635821104049683
# 534

Vocab Size 8933
Accuracy Score  0.841
F1 Score  0.8431952662721893
Precision Score  0.8317120622568094
Recall Score  0.855
2.5710132122039795


1057

In [21]:
unknown_words_test_de = vectorize_predict(w2v_de, X_test_de,y_test_de, LSVC)
len(unknown_words_test_de)

# Vocab Size 7575
# Accuracy Score  0.46508732700791683
# F1 Score  0.46738554854859676
# Precision Score  0.46538604417069107
# Recall Score  0.46940230857557264
# 105.32542681694031
# 100

Vocab Size 11213
Accuracy Score  0.6965
F1 Score  0.7563227619429946
Precision Score  0.6317907444668008
Recall Score  0.942
3.1197590827941895


634

### De-De, De-En, De-Fr

In [22]:
vectorizer = TfidfEmbeddingVectorizer(w2v_de)
vect,vocab = vectorizer.fit(X_unlabeled_de)
X_t = vectorizer.transform(X_unlabeled_de)
clfLSVC = LinearSVC()
LSVC = CalibratedClassifierCV(clfLSVC)
unknown_words_list_de = words_not_w2vec(vocab,w2v_de)
print(len(unknown_words_list_de))

Vocab Size 7575
100


In [23]:
LSVC.fit(X_t, y_unlabeled_de)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid')

In [24]:
unknown_words_test_en = vectorize_predict(w2v, X_test,y_test, LSVC)
len(unknown_words_test_en)

# Vocab Size 6996
# Accuracy Score  0.7691
# F1 Score  0.7481072589618833
# Precision Score  0.8229251668026688
# Recall Score  0.68576
# 35.73703718185425
# 60

Vocab Size 9113
Accuracy Score  0.772
F1 Score  0.7472283813747228
Precision Score  0.8383084577114428
Recall Score  0.674
2.671334981918335


247

In [25]:
unknown_words_test_fr = vectorize_predict(w2v_fr, X_test_fr,y_test_fr, LSVC)
len(unknown_words_test_fr)

# Vocab Size 6261
# Accuracy Score  0.7205962884088835
# F1 Score  0.7501904036557501
# Precision Score  0.6783412858478036
# Recall Score  0.8390629753574688
# 14.586955070495605
# 534

Vocab Size 8933
Accuracy Score  0.709
F1 Score  0.663972286374134
Precision Score  0.7855191256830601
Recall Score  0.575
2.189793109893799


1057

In [26]:
unknown_words_test_de = vectorize_predict(w2v_de, X_test_de,y_test_de, LSVC)
len(unknown_words_test_de)

# Vocab Size 7575
# Accuracy Score  0.81372454221309
# F1 Score  0.8170492114650665
# Precision Score  0.8027221198479159
# Recall Score  0.8318970206079652
# 97.7156879901886
# 100

Vocab Size 11213
Accuracy Score  0.8325
F1 Score  0.8314041268243583
Precision Score  0.8368794326241135
Recall Score  0.826
3.032680034637451


634